__качаем объявления о продаже недвижимости с циан.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
import logging
import sys 

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

In [2]:
from time import sleep
from tqdm.auto import tqdm
import pandas as pd
from datetime import datetime 

pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

---

In [3]:
source = [
    {
            'place': 'sevastopol',
        'subfolder': 'flat',
              'url': 'https://sevastopol.cian.ru/kupit-kvartiru/',
    },{
            'place': 'sevastopol',
        'subfolder': 'land',
              'url': 'https://sevastopol.cian.ru/kupit-zemelniy-uchastok/',
    },{
            'place': 'sevastopol',
        'subfolder': 'house',
              'url': 'https://sevastopol.cian.ru/kupit-dom/',
    },{
            'place': 'sudak',
        'subfolder': 'flat',
              'url': 'https://krym.cian.ru/kupit-kvartiru-krym-sudak/',
    },{
            'place': 'sudak',
        'subfolder': 'land',
              'url': 'https://krym.cian.ru/kupit-zemelniy-uchastok-krym-sudak/',
    },{
            'place': 'sudak',
        'subfolder': 'house',
              'url': 'https://krym.cian.ru/kupit-dom-krym-sudak/',
    },    
#     {
#             'place': 'bahchisaray',
#         'subfolder': 'flat',
#               'url': 'https://krym.cian.ru/kupit-kvartiru-krym-bahchisaray/',
#     }, {
#             'place': 'bahchisaray',
#         'subfolder': 'land',
#               'url': 'https://krym.cian.ru/kupit-zemelniy-uchastok-krym-bahchisaray/',
#     },{
#             'place': 'bahchisaray',
#         'subfolder': 'house',
#               'url': 'https://krym.cian.ru/kupit-dom-krym-bahchisaray/',
#     },

]


---

In [4]:
# from selenium import webdriver
# from lib.cian import CianPageParser

# browser = webdriver.Firefox()
# browser.get(source[0]['url'])
# data = CianPageParser().transform(browser)
# browser.close()

In [5]:
# data
# browser.close()

---

In [6]:
dest_folder = 'data/2024/raw'

In [7]:
from selenium import webdriver
# from selenium.webdriver.common.by import By
from lib.cian import CianParser
# from lib.cian import CianDataCleaner

In [8]:
%%time 

#from selenium.webdriver.firefox.options import Options
#from selenium.webdriver.firefox.options import FirefoxProfile
# profile_path = '/home/mechanoid/.mozilla/firefox/4btdfxw1.default-release'
# options = Options()
# options.profile = FirefoxProfile(profile_path) 
# options.headless = True
# browser = webdriver.Firefox(options=options)

options = webdriver.FirefoxOptions()
options.headless = True
browser = webdriver.Firefox(options=options)
logging.info('open virtual browser')

for s in source:

    url = s['url']
    place = s['place']
    subfolder = s['subfolder'] 
    
    display(url)
    browser.get(url)
    
    data = CianParser().transform(browser)
    assert len(data)>0
    
    df = pd.DataFrame(data)
    df['ts'] = datetime.now()
    df['place'] = place
    
    display( len(df) )
    t_now = datetime.now().strftime('%Y-%m-%d_%H-%M')
    fname = f'{dest_folder}/{subfolder}/cian/cian_{t_now}_raw_{place}.pkl'
    df.to_pickle(fname)
    display( fname )
       
browser.close()

[INFO    ] 2024-09-24 10:32:43 | open virtual browser


'https://sevastopol.cian.ru/kupit-kvartiru/'

[INFO    ] 2024-09-24 10:32:54 | page 1: 28 items
[INFO    ] 2024-09-24 10:33:00 | page 2: 28 items
[INFO    ] 2024-09-24 10:33:07 | page 3: 28 items
[INFO    ] 2024-09-24 10:33:14 | page 4: 28 items
[INFO    ] 2024-09-24 10:33:36 | page 5: 28 items
[INFO    ] 2024-09-24 10:33:41 | page 6: 28 items
[INFO    ] 2024-09-24 10:33:46 | page 7: 28 items
[INFO    ] 2024-09-24 10:33:53 | page 8: 28 items
[INFO    ] 2024-09-24 10:33:59 | page 9: 28 items
[INFO    ] 2024-09-24 10:34:06 | page 10: 28 items
[INFO    ] 2024-09-24 10:34:11 | page 11: 28 items
[INFO    ] 2024-09-24 10:34:16 | page 12: 28 items
[INFO    ] 2024-09-24 10:34:21 | page 13: 28 items
[INFO    ] 2024-09-24 10:34:25 | page 14: 28 items
[INFO    ] 2024-09-24 10:34:30 | page 15: 28 items
[INFO    ] 2024-09-24 10:34:33 | page 16: 28 items
[INFO    ] 2024-09-24 10:34:37 | page 17: 28 items
[INFO    ] 2024-09-24 10:34:41 | page 18: 28 items
[INFO    ] 2024-09-24 10:34:45 | page 19: 28 items
[INFO    ] 2024-09-24 10:34:50 | page 20

895

'data/2024/raw/flat/cian/cian_2024-09-24_10-35_raw_sevastopol.pkl'

...

'https://krym.cian.ru/kupit-dom-krym-sudak/'

[INFO    ] 2024-09-24 10:36:36 | page 1: 28 items
[INFO    ] 2024-09-24 10:36:36 | page 2: 0 items


28

'data/2024/raw/house/cian/cian_2024-09-24_10-36_raw_sudak.pkl'

CPU times: user 8.83 s, sys: 358 ms, total: 9.19 s
Wall time: 3min 55s


In [9]:
# !rm -v data/2024/raw/flat/cian/cian_2024-09-03_13-16_raw_sevastopol.pkl

In [10]:
!date

Вт 24 сен 2024 10:36:36 MSK


In [11]:
# browser.close()